In [1]:
import torch
import torch.nn as nn

In [2]:
# Create a simple linear regression model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # One input feature, one output

    def forward(self, x):
        return self.linear(x)

In [3]:
# Create some dummy data for training
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]], dtype=torch.float32)  # Input data
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]], dtype=torch.float32)  # Corresponding output data

In [4]:
# Instantiate the model
model = LinearRegressionModel()

In [5]:
# Define a loss function and an optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

In [6]:
# Training loop
for epoch in range(1000):
    # Forward pass
    y_pred = model(x)

    # Compute the loss
    loss = criterion(y_pred, y)

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/1000], Loss: {loss.item()}')

Epoch [100/1000], Loss: 0.023748867213726044
Epoch [200/1000], Loss: 0.013037830591201782
Epoch [300/1000], Loss: 0.007157595828175545
Epoch [400/1000], Loss: 0.003929448314011097
Epoch [500/1000], Loss: 0.0021572227124124765
Epoch [600/1000], Loss: 0.0011842898093163967
Epoch [700/1000], Loss: 0.0006501590250991285
Epoch [800/1000], Loss: 0.0003569273976609111
Epoch [900/1000], Loss: 0.00019594885816331953
Epoch [1000/1000], Loss: 0.00010757514974102378


In [7]:
# Save the trained model to a .pth file
torch.save(model.state_dict(), 'linear_regression_model.pth')

In [8]:
print("Model saved to linear_regression_model.pth")

Model saved to linear_regression_model.pth


#------------------------------------------------------------------------

In [9]:
from flask import Flask, request, jsonify
import torch
import torch.nn as nn

In [10]:
app = Flask(__name__)

In [11]:
# Load your trained model
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, x):
        return self.fc(x)

In [29]:
model = RegressionModel()
state_dict = torch.load('/home/abdullah/Documents/linear_regression_model.pth')
# Rename keys to match your model
state_dict['fc.weight'] = state_dict.pop('linear.weight')
state_dict['fc.bias'] = state_dict.pop('linear.bias')
model.load_state_dict(state_dict)
model.eval()

RegressionModel(
  (fc): Linear(in_features=1, out_features=1, bias=True)
)

In [38]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        input_data = torch.tensor(data['input_data'], dtype=torch.float32)
        with torch.no_grad():
            prediction = model(input_data).item()
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(debug=True)

AssertionError: View function mapping is overwriting an existing endpoint function: predict